# Importação das bibliotecas necessárias

In [1]:
import sys
import os
sys.path.append('/home/jovyan/work')

from utils.AlinharETL import AlinharETL

# Crie uma instância da classe AlinharETL

In [2]:
# Crie uma instância da classe
bucket = "silver"
datamart = "SiteMTECbo"
extrator_bronze = AlinharETL(bucket,datamart)

2024-09-25 19:14:45,640 - INFO - Iniciando Sessão Spark.


# Leitura da tabela

In [3]:
df_compliance_cbo_sub_grupo= extrator_bronze.criar_view_temporaria('bronze/Mte/MteCBOSubGrupoPrincipal','compliance_cbo_sub_grupo_principal')

2024-09-25 19:14:51,431 - INFO - Aguarde... Criando View (bronze/Mte/MteCBOSubGrupoPrincipal)
2024-09-25 19:15:18,518 - INFO - View criada com sucesso


In [4]:
df_compliance_cbo_grande_grupo = extrator_bronze.criar_view_temporaria('bronze/Mte/MteCBOGrandeGrupo','compliance_cbo_grande_grupo')

2024-09-25 19:15:18,526 - INFO - Aguarde... Criando View (bronze/Mte/MteCBOGrandeGrupo)


The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


2024-09-25 19:15:19,217 - INFO - View criada com sucesso


# Tratamentos na tabela 

In [5]:
sql_query = """
    SELECT
        sp.cd AS codigo,
        g.demanda_formacao,
        g.grande_grupo,
        sp.TITULO AS subgrupo_principal,
        g.subgrupo,
        g.familia,
        g.ocupacao,
        'SUBGRUPO PRINCIPAL' AS nivel,
        g.industrial_uniepro
    FROM compliance_cbo_sub_grupo_principal sp
    LEFT OUTER JOIN compliance_cbo_grande_grupo g
    ON sp.cd_grande_grupo = g.cd
"""

In [6]:
compliance_flat_ibge_cnae = extrator_bronze.carregar_dados_delta(sql_query)

2024-09-25 19:15:19,233 - INFO - Aguarde... Executando Query (Delta)
2024-09-25 19:15:19,234 - INFO - 
    SELECT
        sp.cd AS codigo,
        g.demanda_formacao,
        g.grande_grupo,
        sp.TITULO AS subgrupo_principal,
        g.subgrupo,
        g.familia,
        g.ocupacao,
        'SUBGRUPO PRINCIPAL' AS nivel,
        g.industrial_uniepro
    FROM compliance_cbo_sub_grupo_principal sp
    LEFT OUTER JOIN compliance_cbo_grande_grupo g
    ON sp.cd_grande_grupo = g.cd

2024-09-25 19:15:19,402 - INFO - Query (Delta) executada com sucesso


# Gravação no datalake

In [7]:
extrator_bronze.caminho_tabela_delta = 's3a://silver/Mte/MteCBOSubGrupoPrincipal'

In [8]:
extrator_bronze.salvar_delta(compliance_flat_ibge_cnae, 'overwrite')

2024-09-25 19:15:19,418 - INFO - Aguarde... Persistindo dados (overwrite)
2024-09-25 19:15:35,080 - INFO - Dados persistidos com sucesso
2024-09-25 19:15:35,081 - INFO - s3a://silver/Mte/MteCBOSubGrupoPrincipal
2024-09-25 19:15:35,082 - INFO - Aguarde... Realizando optimize
2024-09-25 19:15:37,089 - INFO - Optimize executado com sucesso.
2024-09-25 19:15:37,090 - INFO - Aguarde... Realizando vacuum
2024-09-25 19:16:00,374 - INFO - Vacuum executado com sucesso.


# Encerra sessão spark

In [9]:
extrator_bronze.parar_sessao()

2024-09-25 19:16:00,766 - INFO - Sessão Spark finalizada.
